## LAB360: Build an Agentic App with PostgreSQL, GraphRAG, and Semantic Kernel

### Part 3.1: Introduction

**Welcome to the LAB360 Agent App Notebook!**

In this notebook, you will build a Semantic Kernel Agent that can reason over our database of legal cases you deployed in the previous steps. You will also incorporate external web service data, and use memory to improve its responses over time.

Semantic Kernel is an open-source SDK developed by Microsoft that helps developers create advanced AI agents by combining:

- LLMs (Large Language Models) like OpenAI's GPT models
- Plugins (custom tools and functions the agent can call)
- Memory (saving and recalling past conversations or facts)

An Agent in Semantic Kernel is a smart assistant that can:

- Respond to user prompts
- Decide which plugin functions to call
- Use external knowledge sources like databases or APIs
- Build better, grounded answers by combining model reasoning with real-world data

You are about to connect powerful components:

- Azure OpenAI (for embeddings and LLM chat completions)
- PostgreSQL with Vector and Graph extensions (for fast semantic and graph search)
- APIs for real-world data (historical weather evidence)

As you progress, each section of code will incrementally build up these capabilities, and by the final step, you’ll have a highly capable legal research assistant.

### Part 3.2: Setup the Agent App Python imports

> **Note:** In your lab environment, we already have the PIP packages pre-deployed that are needed by the import statements in the following code block, so you do not need to install these.  But just for reference and for future usage of this code, here are the packages used:
> - PostgreSQL connectivity (`psycopg2`, `psycopg-binary`, `psycopg-pool`)
> - Modeling and validation (`pydantic`)
> - OpenAI and Semantic Kernel integration (`openai`, `semantic-kernel`)
> - Notebook compatibility (`nest_asyncio`, `ipykernel`)

##### 🧠 *Technical Notes*

This set of imports prepares the technical foundation for building an AI-powered agent that interacts with a PostgreSQL database and OpenAI services. `nest_asyncio` is used to allow nested event loops, which is important when running asynchronous code inside a Jupyter notebook. Core Python modules like `os`, `asyncio`, `uuid`, and `requests` handle environment access, asynchronous execution, unique ID generation, and external API calls, respectively. `psycopg2` provides database connectivity to PostgreSQL, while `pydantic` offers structured data validation and modeling.

The Semantic Kernel libraries enable creation of intelligent agents (`ChatCompletionAgent`), define plugins and function tools (`kernel_function`), manage prompt settings, and handle retrieval-augmented memory through `PostgresMemoryStore` and `SemanticTextMemory`. Finally, Azure OpenAI integration components (`AzureChatCompletion`, `AzureTextEmbedding`) allow the agent to generate responses and embeddings using cloud-based AI models.

##### ⚙️ *Code Review Tasks*

1. Run the cell below using the "▶" icon next to the cell.

1. This will run the code and show the output below.  Since these are just imports, there is nothing to show at the end other than a check mark showing success.

    > **Note:** The first time this code block is ran, it may take around 20-30 seconds.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import os
import asyncio
import psycopg2
import uuid
import requests
from typing import Annotated
from pydantic import BaseModel

from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatPromptExecutionSettings
from semantic_kernel.functions import kernel_function, KernelArguments

from semantic_kernel import Kernel
from semantic_kernel.connectors.memory.postgres import PostgresMemoryStore
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.connectors.ai.open_ai.services.azure_text_embedding import AzureTextEmbedding

### Part 3.3: Setup environmental connection variables

##### 🧠 *Technical Notes*

In order for our agent to interact with both Azure OpenAI services and the PostgreSQL database, we need to configure a few environment-specific connection variables. Rather than manually retrieving these values from the Azure Portal, we provide a shortcut: simply run the script located at `./Scripts/get_env.ps1` to automatically extract the correct credentials. After running the script, copy the output values into the fields below.

This setup allows the agent to securely authenticate API requests to Azure OpenAI (using `AZURE_OPENAI_*` variables) and connect to the database where our case law data is stored (using the `DB_CONFIG` dictionary).

##### ⚙️ *Code Review Tasks*

1. Within VS Code, open a new terminal, and at the following path, enter:

    `PS C:\Users\LabUser\Downloads\pg-sk-agents-lab> .\Scripts\get_env.ps1`

1. From the output of the script in the terminal, copy the values for the following each into the variables in the code block below:
    - `AZURE_OPENAI_ENDPOINT`
    - `AZURE_OPENAI_KEY`
    - `DB_CONFIG - HOST`
    - `DB_CONFIG - PASSWORD`

    > **Note:** For `DB_CONFIG - PASSWORD`, this is a very long string due to being an Entra ID Access Token, be sure to copy the entire string as the password.

1. For the value for `{USER}`, this will be your Lab Username Credential, you can get these from the Lab Manual, in Part 3.3

1. Once you have each of these fields filled in, then run the cell below using the "▶" icon next to the cell.  This will run the code and show the output below.  You will reuse these variables and objects throughout the lab notebook below.

1. You should see a check mark when it completes, showing success.

In [ ]:
AZURE_OPENAI_ENDPOINT   = "{AZURE_OPENAI_ENDPOINT}"
AZURE_OPENAI_KEY        = "{AZURE_OPENAI_KEY}"
AZURE_OPENAI_DEPLOYMENT = "gpt-4o"

DB_CONFIG = {
    "host":     "{HOST}",
    "dbname":   "cases",
    "user":     "{USER}",
    "password": "{PASSWORD}",
    "port":     5432,
    "sslmode":  "require"
}

### Part 3.4: Create Semantic Kernel Plugin for Basic Database Queries

##### 🧠 *Technical Notes*

In this step, we create a custom plugin called DatabaseSearchPlugin to give our agent the ability to interact directly with the case law database using basic SQL queries. This plugin uses the psycopg2 library to establish a connection to PostgreSQL, execute queries, and return results. We define two simple but important functions: `count_cases()` to return the total number of cases in the database, and `search_cases(keyword)` to perform a keyword search against case opinions. Each function is decorated with `@kernel_function`, which registers it as a callable tool within the Semantic Kernel framework. This makes these database operations available to the agent automatically during conversation, enabling the agent to retrieve real-time, grounded information from our dataset.

##### ⚙️ *Code Review Tasks*

1. Review the code below, notice the print statements outputting to the terminal the name of the function when it is called.  This will be helpful later when we run the agent, and we want to see what functions it chose to call.

1. Run the cell below using the "▶" icon next to the cell.

1. This will run the code and show the output below.  Since these are just imports, there is nothing to show at the end other than a check mark showing success.

In [89]:
class DatabaseSearchPlugin:
        def __init__(self, cfg):
            self.cfg = cfg

        @kernel_function(description="Return the total number of cases in the database.")
        def count_cases(self) -> str:
            
            # print statement so we can see when this function chosen by the agent and is called
            print("count_cases was called")
            
            conn = psycopg2.connect(**self.cfg)
            cur = conn.cursor()
            cur.execute("SELECT COUNT(*) FROM cases;")
            n = cur.fetchone()[0]
            conn.close()
            return str(n)

        @kernel_function(description="Find up to 10 case IDs and names whose opinion contains the given keyword.")
        def search_cases(self, keyword: str) -> str:
            
            # print statement so we can see when this function chosen by the agent and is called
            print("search_cases was called")
            
            conn = psycopg2.connect(**self.cfg)
            cur = conn.cursor()
            cur.execute(
                "SELECT id, name, opinion FROM cases WHERE opinion ILIKE %s LIMIT 10;",
                (f"%{keyword}%",)
            )
            rows = cur.fetchall()
            conn.close()
            if not rows:
                return "No matches"
            return "\n".join(f"{r[0]}: {r[1]}: {r[2][:1000]}" for r in rows)

### Part 3.5: Test Run of our New Agent

##### 🧠 *Technical Notes*

Now that we have created our first plugin, we're ready to assemble and test an initial version of our agent. In this step, we create a default `OpenAIChatPromptExecutionSettings` to define basic settings. We then create an instance of `AzureChatCompletion`, which serves as the underlying communication layer between our agent and Azure OpenAI. Using these components, we instantiate a `ChatCompletionAgent`, providing it a name, a set of behavioral instructions, and a list of available plugins (in this case, just `DatabaseSearchPlugin`).

Finally, we send a sample user query to the agent and retrieve its response. This test validates that our agent can successfully invoke plugin functions, query the database, and integrate the results into a natural language reply.

##### ⚙️ *Code Review Tasks*

1. Review the code below, notice the following:
    - Inside `ChatCompletionAgent` we define `instructions` which act as notion of a "system prompt" for the Agent to define its purpose and goals
    - For now we are passing in our `DatabaseSearchPlugin` class, we will be creating more PlugIns to enhance the functionality of our agent in the next steps
    - Notice the `user_query` variable, and how it is asking how many cases there are, plus about water leaking cases.

1. Run the cell below using the "▶" icon next to the cell.  This invokes the agent and subsequent LLM calls, this may take a few moments to run.

1. After the code runs, notice the following:
    - There should have been 2 functions called: `count_cases` and `search_cases`
    - This happened because the agent interpreted the prompt and made a decision to call these 2 functions
    - Notice how we got back a clear response of 377 cases are in our database.  This was based on our `count_cases` database function giving the LLM grounded truth about our dataset.
    - Lastly, notice how we asked for 10 cases, but only got 2.  This is because our `search_cases` function is just the ILIKE operator and not yet using a vector search.  It could only find 2 cases that matching using the basic keyword ILIKE search.  In our next lab sections, we will see how we can improve on this.

In [ ]:
settings = OpenAIChatPromptExecutionSettings()
    
chat_svc = AzureChatCompletion(
    deployment_name=AZURE_OPENAI_DEPLOYMENT,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY)

# Create agent with plugin and settings
agent = ChatCompletionAgent(
    service=chat_svc,
    name="SK-Assistant",
    instructions="You are a helpful legal assistant.  Respond to the user with the name of the cases and reasoning why the cases are the most relevant, and a short sentence summary of the opinion of the cases.",
    plugins=[DatabaseSearchPlugin(DB_CONFIG)],
    arguments=KernelArguments(settings)
)

user_query = "How many cases are there, and find me 10 cases regarding the notion of water leaking."

print("// Functions the Agent Called: //")

response = await agent.get_response(messages=user_query)

print("// Agent Response: //")
print(response.content)


### Part 3.6: Improve Agent Accuracy by Adding Semantic Re-ranking Query Plugin

##### 🧠 *Technical Notes*

In this step, we add a new plugin called `SemanticRerankingPlugin` to increase the precision of our agent’s search results. Instead of relying only on keyword matching, this plugin uses semantic search and re-ranking to evaluate results based on the meaning and intent of the user query.

It works in two phases: first, it performs a vector similarity search using Azure OpenAI embeddings to find approximately relevant cases; then, it reorders these using a separate model (e.g., `semantic_relevance`) that scores and ranks the results for deeper semantic alignment. This two-step approach helps the agent prioritize results that are not just textually similar, but also topically and contextually relevant—making it ideal for legal queries where nuance matters. As with other tools, this function is registered using `@kernel_function`, so the agent can intelligently decide when to use it.

##### ⚙️ *Code Review Tasks*

1. Before running this code block, we need to run a SQL script on the database.
    - In VS Code, in the folder structure, in the folder `./Scripts/` navigate to the `setup_reranker.sql` file
    - Once inside the file, still in VS Code, press on the keyboard `CTRL+SHIFT+P` to open the VS Code action panel, type `PGSQL Connect` and select the top `PGSQL: Connect` option
    - Once prompted, select the Connection you made in the earlier steps in the lab
    - You will then be asked the port number for the Connection, just hit `enter` to accept the default
    - You should now be Connected to your database in the `setup_reranker.sql` file
    - Next we need to replace the `{ENDPOINT}` and `{KEY}` tokens with the real values.  You can get these in the Skillable Lab Manual, on Part 3.6
    - In the editor window, click the ">" button in the top right to run the script
    - This will setup our connection to Azure ML via our `azure_ai` PostgreSQL database extension, and create 2 PostgresSQL PL/SQL functions needed for semantic re-ranking

1. Review the code:
    - Notice the following enhancements:
        - This plugin introduces a function called `search_semantic_reranked_cases`, designed to deliver higher-quality search results by understanding the semantic meaning of the query rather than relying on basic keyword matches.
        - The function is decorated with `@kernel_function`, which makes it available to the agent as a callable tool-this is how Semantic Kernel enables function calling automatically based on user intent.

    - Check out the SQL logic inside the code, notice how it includes two key phases:
        - (Phase 1) A vector similarity search using OpenAI-generated embeddings to find top 60 candidate cases.
        - (Phase 2) A semantic re-ranking step that uses an external re-ranker model (semantic_relevance) to evaluate and reorder those 60 based on relevance scores.
        - This hybrid approach helps ensure that even when the user’s phrasing doesn’t exactly match the database text, relevant results can still be surfaced based on meaning and context.

    - Observe how this plugin builds on the earlier keyword search:
        - The previous `search_cases()` method used `ILIKE` for fuzzy keyword matching. That works for exact or near-exact phrases but misses nuance.
        - This plugin improves accuracy and recall, especially for complex queries or abstract legal concepts where keyword overlap may be weak.
        - This function will be automatically called by the agent if the prompt includes phrases like "high accuracy is important" or contains complex, open-ended search intent.
        - For example, a prompt like: "*Help me find the most relevant cases about tenant water damage, with high accuracy*" will likely trigger this plugin over the basic one.

1. Finally, run the code block cell by clicking the "▶" button on the left side of the code block.
    - This will run the code but there will be no output yet.
    - Since this is just a class definition, there is nothing to show at the end other than a check mark showing success.

In [ ]:
class SemanticRerankingPlugin:
        def __init__(self, cfg):
            # Store the database configuration for later use
            self.cfg = cfg

        @kernel_function(description="Use semantic re-ranking function to query and find cases matching the query based on semantic intent and relevance.  Use this function when high accuracy is needed.")
        def search_semantic_reranked_cases(self, query: str) -> str:
            
            # Log when the function is invoked
            print("search_semantic_reranked_cases was called")
            
            # Connect to the PostgreSQL database
            conn = psycopg2.connect(**self.cfg)
            cur = conn.cursor()

            # Execute a SQL query that performs semantic vector search and re-ranking
            cur.execute(
                """
                -- // Step 1: Create an embedding from the user's query using Azure OpenAI //
                WITH embedding_query AS (
                    SELECT azure_openai.create_embeddings('text-embedding-3-small', %s)::vector AS embedding
                ),

                -- // Step 2: Find top 60 cases whose vector is closest to the query embedding (initial vector search) //
                vector AS (
                    SELECT cases.id as case_id, cases.name AS case_name, cases.opinion, RANK() OVER (ORDER BY opinions_vector <=> embedding) AS vector_rank
                    FROM cases, embedding_query
                    ORDER BY opinions_vector <=> embedding
                    LIMIT 60
                ),

                -- // Step 3: Call a semantic re-ranking function to evaluate the 60 candidates and assign relevance scores //
                semantic AS (
                    SELECT * 
                    FROM jsonb_array_elements(
                            semantic_relevance(%s, 60)
                        ) WITH ORDINALITY AS elem(relevance)
                ),

                -- // Step 4: Join the initial vector results with semantic scores and rank by highest relevance //
                semantic_ranked AS (
                    SELECT semantic.relevance::DOUBLE PRECISION AS relevance, RANK() OVER (ORDER BY relevance DESC) AS semantic_rank,
                            semantic.*, vector.*
                    FROM vector
                    JOIN semantic ON vector.vector_rank = semantic.ordinality
                    ORDER BY semantic.relevance DESC
                )

                -- // Step 5: Return the top 10 results after semantic re-ranking //
                SELECT case_id, case_name, opinion
                FROM semantic_ranked
                LIMIT 10;
                """, (query, query)) # Pass the query twice: once for embedding, once for semantic re-ranker

            rows = cur.fetchall() # Fetch the final ranked results

            conn.close() # Always close the connection after use

            # Return a nicely formatted list of top results or fallback message if no matches
            if not rows:
                return "No matches"
            return "\n".join(f"{r[0]}: {r[1]}: {r[2][:1000]}" for r in rows) # Truncate long opinions to 1000 chars

### Part 3.7: Add a GraphRAG Query PlugIn to the Agent for Additional Accuracy Improvements

##### 🧠 *Technical Notes*

In this step, we build another advanced plugin, `GraphDatabasePlugin`, that combines vector search with graph analysis to find the most influential cases related to a query topic. The `search_graph_cases` method first uses an embedding-based similarity search to semantically rank cases from the cases table. This ensures that only cases meaningfully related to the query are considered further.

After narrowing the results semantically, the query leverages **Apache AGE** - a PostgreSQL extension that enables property graph querying via Cypher syntax. Specifically, it matches citations (relationships between cases) in the `case_graph` graph. By counting the number of incoming edges (citations) for each case, we can rank cases based on their influence or importance within the graph. Cases with more citations are prioritized, resulting in a more nuanced retrieval that considers both semantic relevance and citation authority.

This hybrid retrieval technique is an example of **GraphRAG (Graph Retrieval-Augmented Generation)** and represents a more sophisticated form of grounded information retrieval for legal, academic, or research applications.

##### ⚙️ *Code Review Tasks*

1. Before running this code block, we need to run a SQL script on the database.
    - In VS Code, in the folder structure, in the folder `./Scripts/` navigate to the `create_graph.sql` file
    - Once inside the file, still in VS Code, press on the keyboard `CTRL+SHIFT+P` to open the VS Code action panel, type `PGSQL Connect` and select the top `PGSQL: Connect` option
    - Once prompted, select the Connection you made in the earlier steps in the lab
    - You will then be asked the port number for the Connection, just hit `enter` to accept the default
    - You should now be Connected to your database in the `create_graph.sql` file
    - In the editor window, click the ">" button in the top right to run the script
    - This will build our graph database via the Apache AGE extension in our Azure PostgreSQL database using our loaded 377 legal cases

1. Next, because we are using the Apache AGE PostgreSQL Extension to provide us Graph database capabilities, we need to enable the Extension on our database.
    - Run the following PowerShell script within VS Code
    - Within VS Code, open a new terminal, and at the following path, enter:

        `PS C:\Users\LabUser\Downloads\pg-sk-agents-lab> .\Scripts\load_age.ps1`

        > **Note:** This will run through 3 main commands, all together will take around 60-120 seconds.

1. Review the code:
    - This plugin allows the agent to find legal cases that are not only semantically similar to a user's query but also highly cited by other cases - providing both relevance and legal importance. The `@kernel_function` decorator makes this method callable by the agent.
    - Look at the `semantic_ranked` CTE (Common Table Expression). This ranks cases by their similarity to the input query using embedding-based vector comparison (<=>). The function limits results to the top 60 most semantically similar opinions using Azure OpenAI’s embedding model.
    - Examine the graph CTE. It runs a Cypher query on the `case_graph` to count how many times each case is cited by others. These citation counts are joined to the semantic results using the case ID. This allows the plugin to prioritize not just relevant cases, but those that are also influential in the citation network.
    - The final `SELECT` returns 10 cases with the highest number of citations among the semantically relevant ones. The list is ordered by refs `DESC`, meaning more citations come first. Each opinion is truncated to the first 1000 characters to keep responses concise.

1. Finally, run the code block cell by clicking the "▶" button on the left side of the code block.
    - This will run the code but there will be no output yet.
    - Since this is again just a class definition, there is nothing to show at the end other than a check mark showing success.

In [ ]:
class GraphDatabasePlugin:
        def __init__(self, cfg):
            # Store the database configuration dictionary
            self.cfg = cfg

        @kernel_function(description="Use an advanced accuracy query to find important cases with high levels of citations about the query topic.")
        def search_graph_cases(self, query: str) -> str:
            
            # Debug log to indicate function was triggered
            print("search_graph_cases was called")
            
            # Connect to the PostgreSQL database using the provided config
            conn = psycopg2.connect(**self.cfg)
            cur = conn.cursor()

            # Execute a compound query using semantic search + graph analysis (Apache AGE)
            cur.execute(
                """
                -- // Step 1: Set the schema search path to include Apache AGE graph catalog //
                SET search_path = public, ag_catalog, "$user";

                -- // Step 2: Rank cases semantically using embedding similarity //
                WITH semantic_ranked AS (
                    SELECT id, name, opinion, opinions_vector
                    FROM cases
                    ORDER BY opinions_vector <=> azure_openai.create_embeddings('text-embedding-3-small', %s)::vector
                    LIMIT 60
                ),
                -- // Step 3: Use a Cypher graph query to count how many citations (edges) each case has //
                graph AS (
                    SELECT graph_query.refs, semantic_ranked.*, graph_query.case_id 
                    FROM semantic_ranked
                    LEFT JOIN cypher('case_graph', $$
                        MATCH ()-[r]->(n)
                        RETURN n.case_id, COUNT(r) AS refs
                    $$) as graph_query(case_id TEXT, refs BIGINT)
                    ON semantic_ranked.id = graph_query.case_id::int
                )
                -- // Step 4: Return the top 10 cases, prioritized by number of citations (refs) //
                SELECT id, name, opinion
                FROM graph
                ORDER BY refs DESC NULLS LAST
                LIMIT 10;
                """, 
                (f"%{query}%",)
            )
            rows = cur.fetchall() # Fetch all resulting rows
            conn.close() # Close the database connection

            # Return either "No matches" or a formatted list of results
            if not rows:
                return "No matches"
            return "\n".join(f"{r[0]}: {r[1]}: {r[2][:1000]}" for r in rows) # Truncate long opinions to 1000 chars

### Part 3.8: Re-Assemble our Agent with new advanced PlugIns and Re-Test

##### 🧠 *Technical Notes*

In this step, we re-assemble the full agent by attaching all of the custom plugins we’ve created so far: `DatabaseSearchPlugin`, `SemanticRerankingPlugin`, and `GraphDatabasePlugin`. These plugins give the agent access to different querying strategies, from simple keyword searches to advanced semantic filtering and graph-based citation analysis. By registering all plugins in the plugins list, we enable the agent to choose the right tool based on the intent expressed in the user’s prompt.

##### ⚙️ *Code Review Tasks*

1. Review the code below, notice the following:
    - We only need to re-define our agent object using the `ChatCompletionAgent` class
    - Notice we are now adding our new PlugIns on the line:
        - `plugins=[DatabaseSearchPlugin(DB_CONFIG),SemanticRerankingPlugin(DB_CONFIG),GraphDatabasePlugin(DB_CONFIG)],`    
    - Notice the `user_query` variables, there are some additional ones you can try testing yourself by uncommenting one at a time, then re-running the code cell

1. Run the cell below using the ">" icon next to the cell.  This invokes the agent and subsequent LLM calls, this may take a few moments to run.

1. After the code runs, notice the following:
    - Depending on the prompt you chosen, there should have been between 2-3 functions called, such as: `search_graph_cases` and `search_semantic_reranked_cases`
    - Notice how we asked for 10 cases, and now got 10 cases. This is because the we are now using semantic vector search, not just keyword search for the database queries.

In [ ]:
# Re-Create agent with new plugins
agent = ChatCompletionAgent(
    service=chat_svc,
    name="SK-Assistant",
    instructions="You are a helpful legal assistant.  Respond to the user with the name of the cases and reasoning why the cases are the most relevant, and a short sentence summary of the opinion of the cases.",
    plugins=[DatabaseSearchPlugin(DB_CONFIG),SemanticRerankingPlugin(DB_CONFIG),GraphDatabasePlugin(DB_CONFIG)],
    arguments=KernelArguments(settings)
)

# Try these different prompts to see how the agent responds:
# Remove one comment at a time to test different prompts
user_query = "Help me find 10 highly relevant cases related to water leaking in my personal home apartment from the floor above.  High accuracy is important, and high number of citations is important.  Also how many cases are there overall?"
#user_query = "Bring into 1 list of 10 cases, ranked by relevancy -- Help me find 10 highly relevant cases related to water leaking in my personal home apartment from the floor above.  High accuracy is important, and high number of citations is important."
#user_query = "How many cases are there, and high accuracy is important, help me find 10 highly relevant cases related to water leaking in my apartment."

print("// Functions the Agent Called: //")

response = await agent.get_response(messages=user_query)

print("// Agent Response: //")
print(response.content)

### Part 3.9: Adding a Weather PlugIn to the Agent

##### 🧠 *Technical Notes*

To enhance our legal assistant with real-world context, we introduce a `WeatherPlugin` that enables the agent to retrieve historical weather data (specifically rainfall) based on a given date and geographic location. This is especially useful in real estate or tenant-landlord disputes where weather-related damage (e.g., leaks or flooding) may be a legal factor. The plugin uses the Open-Meteo Archive API, a free and reliable weather data service.

When a user prompt mentions a need for weather data—such as "What was the rainfall on Feb 1, 2024, in Seattle?"—the agent will automatically call this function. The returned data provides accurate, grounded evidence that enhances the agent’s response and credibility.

##### ⚙️ *Code Review Tasks*

1. Review the code below and observe the following:
    - The `@kernel_function` decorator registers this function so the agent can call it based on user intent.
    - The plugin uses `requests.get()` to make a live API call to Open-Meteo, passing latitude, longitude, and date parameters.
    - The response is parsed from JSON and extracts the precipitation value from the `daily.precipitation_sum` array.
    - The result is returned as a readable string with the date, coordinates, and rainfall in millimeters.

1. Run the cell below using the "▶" (Run) button. There is no visible output until the function is called by the agent in a relevant prompt.

In [ ]:
class WeatherPlugin:
    def __init__(self):
        pass

    @kernel_function(
        description="Get total precipitation (in mm) on a given date and location (latitude, longitude)."
    )
    def get_historical_rainfall(self, date: str, latitude: float, longitude: float) -> str:
        """
        date: YYYY-MM-DD
        latitude, longitude: WGS84 coords
        """

        print("get_historical_rainfall was called")

        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "start_date": date,
            "end_date":   date,
            "daily":      "precipitation_sum",
            "timezone":   "UTC"
        }
        resp = requests.get(url, params=params)
        resp.raise_for_status()

        data = resp.json()
        # the API returns a list for daily.precipitation_sum
        rain_mm = data["daily"]["precipitation_sum"][0]
        return f"On {date} at ({latitude}, {longitude}), total precipitation was {rain_mm} mm."

### Part 3.10: Add our new Weather PlugIn to our Agent and Re-Test

##### 🧠 *Technical Notes*

In this step, we complete our agent by including the new `WeatherPlugin` alongside our database and semantic plugins. This enables the agent to answer more complex, multi-part prompts that require both legal case analysis and external factual grounding—such as historical rainfall on a specific date and location.

When a user prompt mentions weather-related conditions (e.g., "*What was the rainfall on February 1, 2024, in Seattle?*"), the agent can automatically call the appropriate plugin function. Semantic Kernel handles function selection based on the natural language intent, so all tools are available simultaneously without manual switching. This demonstrates the power of multi-plugin orchestration in agent design—allowing a single agent to reason across legal data and real-world APIs in one coherent response.

##### ⚙️ *Code Review Tasks*

1. Review how the `WeatherPlugin()` is added to the list of available plugins in the `ChatCompletionAgent`.

1. Examine the user_query. Note how it includes both:
    - A factual request (weather evidence),
    - And a legal research task (finding relevant legal cases).

1. Run the cell below using the "▶" (Run) button and observe:
    - Which functions the agent chooses to call (check your logs or printed output),
    - How the agent combines different results into a single response.
    - Try editing the user_query to include other dates or cities and observe how the weather data changes.

In [ ]:
# Re-Create agent with new plugins
agent = ChatCompletionAgent(
    service=chat_svc,
    name="SK-Assistant",
    instructions="You are a helpful legal assistant.  Respond to the user with the name of the cases and reasoning why the cases are the most relevant, and a short sentence summary of the opinion of the cases.",
    plugins=[DatabaseSearchPlugin(DB_CONFIG),SemanticRerankingPlugin(DB_CONFIG),GraphDatabasePlugin(DB_CONFIG),WeatherPlugin()],
    arguments=KernelArguments(settings)
)

# Try these different prompts to see how the agent responds:
# Remove one comment at a time to test different prompts
user_query = "What was the rainfall on 2024-02-01 in Seattle, WA? I need this for evidence.  Also High accuracy is important, help me find 10 highly relevant cases related to water leaking in my clients apartment."
#user_query = "I am a real estate lawyer, so cases need to be related to real estate law."
#user_query = "Find me 10 cases regarding the notion of water leaking."
#user_query = "How many cases are there, and high accuracy is important, help me find 10 highly relevant cases related to water leaking in my apartment."
#user_query = "Help me find 10 highly relevant cases related to water leaking in my personal home apartment from the floor above.  High accuracy is important, and high number of citations is important.  Also how many cases are there overall?"
#user_query = "Bring into 1 list of 10 cases, ranked by relevancy -- Help me find 10 highly relevant cases related to water leaking in my personal home apartment from the floor above.  High accuracy is important, and high number of citations is important."
#user_query = "How many cases are in my database? What was the rainfall on 2024-02-01 in Seattle, WA? I need this for evidence.  Also High accuracy is important, help me find 10 highly relevant cases related to water leaking in my clients apartment."

print("// Functions the Agent Called: //")
response = await agent.get_response(messages=user_query)

print("// Agent Response: //")
print(response.content)

### Part 3.11: Adding Memory into the Agent

##### 🧠 *Technical Notes*

In this final step, we complete our agent’s capabilities by enabling semantic memory using SemanticTextMemory backed by a PostgreSQL vector store. This allows the agent to store and recall relevant prior user queries and its own responses—creating a more personalized and context-aware experience over time.

When a new prompt is received, the agent retrieves semantically similar past queries from memory using embedding-based vector search. These memories are then prepended to the current prompt, giving the agent important context and continuity across interactions. This memory is especially useful in real-world legal scenarios where a user may build a case over several prompts, and the agent must retain prior details to offer more precise, relevant guidance.

##### ⚙️ *Code Review Tasks*

1. Review how PostgresMemoryStore and AzureTextEmbedding are used to set up vector-based memory for the agent.

1. Examine how the user query is stored into memory, and how top related queries are retrieved with semantic search.

1. Observe how the memory context is prepended to the new prompt before being passed to the agent.

1. Run the cell below using the "▶" (Run) button and inspect:
    - How the agent’s output incorporates the memory context,
    - What gets saved to memory as both the query and the agent’s reply,
    - How this memory layer enables continuity in multi-turn conversations.

1. Test running through the first 3 sample prompts below, by uncommenting a single prompt, one at a time
    - As you run through these, this helps to showcase the memory functionality as the Agent keeps track of these facts

In [ ]:
# Try running the following 3 prompts, one at a time, this helps to showcase the memory functionality as the Agent keeps track of these facts
#user_query = "I am a real estate lawyer"
#user_query = "My client has lived in their apartment for 10 years"
#user_query = "How many cases are there, and high accuracy is important, help me find 10 highly relevant cases related to water leaking in my apartment."

# Additional test cases to see how the agent responds:
#user_query = "Find me some cases regarding the notion of my house falling down."
#user_query = "Find me 10 cases regarding the notion of water leaking."
#user_query = "Help me find 10 highly relevant cases related to water leaking in my personal home apartment from the floor above.  High accuracy is important, and high number of citations is important.  Also how many cases are there overall?"
#user_query = "Bring into 1 list of 10 cases, ranked by relevancy -- Help me find 10 highly relevant cases related to water leaking in my personal home apartment from the floor above.  High accuracy is important, and high number of citations is important."

# Construct the PostgreSQL connection string for use with memory storage
conn_str = (
f"host={DB_CONFIG['host']} "
f"port={DB_CONFIG['port']} "
f"dbname={DB_CONFIG['dbname']} "
f"user={DB_CONFIG['user']} "
f"password={DB_CONFIG['password']} "
f"sslmode={DB_CONFIG['sslmode']}"
)

# Initialize a Postgres-backed memory store for saving and retrieving vector-based memories
memory_store = PostgresMemoryStore(
    connection_string=conn_str,
    default_dimensionality=1536
)

# Set up the Azure OpenAI embedding service to convert text into vector embeddings
embedding_generator = AzureTextEmbedding(
    deployment_name="text-embedding-3-small",
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY
)

# Combine storage and embedding logic into a semantic memory system
semantic_memory = SemanticTextMemory(
    storage=memory_store,
    embeddings_generator=embedding_generator
)

# Save the user query into memory
await semantic_memory.save_information(
    collection="agent_memories",    # Memory collection name
    text=user_query,                # Text content to embed and store
    id=str(uuid.uuid4()),           # Unique ID for this memory entry        
    description="User query"        # Optional label or tag for this entry
)

# Search memory for the top-3 most semantically similar past queries
recalls = await semantic_memory.search(
    collection="agent_memories",
    query=user_query,
    limit=3
)

# Format the recalled memories as bullet points for context injection
memory_context = "\n".join(f"- {m.text}" for m in recalls)

# Construct a new prompt that includes relevant memory context followed by the current query
prompt = (
    f"Here are things we’ve discussed before:\n{memory_context}\n\n"
    f"{user_query}"
)

print("// Functions the Agent Called: //")
response = await agent.get_response(messages=prompt)

print("// Prompt with Added Memory Context: //")
print(prompt)

print("// Agent Response: //")
print(response.content)

# Save the agent's reply to memory as well, completing a full query-response memory cycle
await semantic_memory.save_information(
    collection="agent_memories",
    text=str(response.content),
    id=str(uuid.uuid4()),
    description="Agent reply")

## Congratulations you have completed the Lab!